In [ ]:
def get_token():
    file_name = input('Введите название файла с токеном с РАСШИРЕНИЕМ. При пустом вводе будет производиться поиск файла с моим дефолтным названием.')
    if file_name:
        file = open(file_name, 'r', encoding='utf-8')
    else:
        file = open('verbindung.txt', 'r', encoding='utf-8')
    token = file.read()
    return token

Соответственно, сам файл скрыт гитигнором

In [ ]:
import json
import urllib.request

def get_users():
    userset = set()
    print("Вводите по одному нику. Пустой код будет окончать завершение пополнения списка.\n")
    print('NB: если вы не введете ни одного пользователя, вам будет предложен заранее составленный список.\n')
    while True:
        user = input("Введите ник пользователя: ")
        if user:
            userset.add(user)
        else:
            break
    if userset != set():
        print("\nИтоговый вид вашего списка:\n", list(userset))
        return userset
    else:
        print("\nВы не внесли в список никого, поэтому будете работать с нашим списком:")
        default_userlist = ['ancatmara', 'JelteF', 'bcongdon', 'arsenitheunicorn']
        print(default_userlist)
        return set(default_userlist)

In [ ]:
def choose_user(userset):
    user = 'костыль'
    while user not in userset:
        user = input('Выберите пользователя и введите его ник: ')
    print("\nВы выбрали пользователя", user, '\n')
    return user

In [ ]:
def download_data(user, token):
    page = 0
    data = list()
    while len(data) == 100 * page:
        page += 1
        url = 'https://api.github.com/users/%s/repos?page=%s&per_page=100&access_token=%s' % (user, page, token)
        jason = urllib.request.urlopen(url)
        text = jason.read().decode('utf-8')
        data += json.loads(text)
    return data

In [ ]:
def send_reps(data):
    reps = {}
    for rep in data:
        reps[rep['name']] = rep['description']
    print("Список его репозиториев:\n")
    for name in reps:
        if reps[name]:
            print(name, ':\t', reps[name], '\n')
        else:
            print(name, ':\t(((описание отсутствует)))\n')

In [ ]:
def languages(data, user):
    lang_dict = dict()
    for rep in data:
        if rep['language']:
            if rep['language'] in lang_dict:
                lang_dict[rep['language']] += 1
            else:
                lang_dict[rep['language']] = 1
    answer = 'Языки, на которых пишет пользователь %s: ' % user
    for lang in sorted(lang_dict, key=lang_dict.get, reverse=True):
        lang_card = lang + ' (%s), ' % str(lang_dict[lang])
        answer += lang_card
    print(answer[:len(answer) - 2])  ## убираем лишние ", "
    print('\nNB: В скобках дано число репозиториев, в которых используется этот язык')
    return lang_dict

In [ ]:
def more_reps(userset, token):
    wb = dict()
    for person in userset:
        wb[person] = len(download_data(person, token))
    winner = sorted(wb, key=wb.get, reverse=True)[0] 
    print("\nСреди пользователей", list(userset), "больше всего репозиториев у пользователя", winner)

про перменную winner:
делаем список из значений словаря, ранжированный по значениям (.get) в порядке убывания (reverse=True)

In [ ]:
def popular_lang(userset, token):
    lang_rating = dict()
    for person in userset:
        all_users_reps = download_data(person, token)
        for rep in all_users_reps:
            if rep['language']:
                if rep['language'] in lang_rating:
                    lang_rating[rep['language']] += 1
                else:
                    lang_rating[rep['language']] = 1
    winner = sorted(lang_rating, key=lang_rating.get, reverse=True)[0]
    print("\nСамый популярный язык среди пользователей из списка —", winner)

In [ ]:
def followers(userset, token):
    foll_rating = dict()
    for person in userset:
        fllwrs = list()
        page = 0
        while len(fllwrs) == 100 * page:
            page += 1
            url = 'https://api.github.com/users/%s/followers?page=%s&per_page=100&access_token=%s' % (person, page, token)
            jason = urllib.request.urlopen(url)
            text = jason.read().decode('utf-8')
            fllwrs += json.loads(text)
        foll_rating[person] = len(fllwrs)
    winner = sorted(foll_rating, key=foll_rating.get, reverse=True)[0]
    print("\nБольше всего фолловеров у", winner)

In [ ]:
def main():
    token = get_token()
    userset = get_users()
    user = choose_user(userset)
    data = download_data(user, token)
    send_reps(data)
    lang_dict = languages(data, user)
    more_reps(userset, token)
    popular_lang(userset, token)
    followers(userset, token)

In [ ]:
if __name__ == '__main__':
    main()